# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

(trainX, trainY),(testX, testY) = tf.keras.datasets.cifar10.load_data()


TensorFlow 2.x selected.
170500096/170498071 [==============================] - 2s 0us/step


In [0]:
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import load_model

## **Dependencies along with all the data is loaded to the notebook**


In [3]:
np.shape(trainY)

(50000, 1)

In [4]:
np.shape(trainX)


(50000, 32, 32, 3)

In [5]:
p = trainY>=5

p = p.reshape(50000,)
p.shape

(50000,)

In [6]:
trainX59 = trainX[p,:,:,:] 
trainX59.shape

(25000, 32, 32, 3)

In [7]:
trainY59 = trainY[p,:]
trainY59.shape

(25000, 1)

In [8]:
p = testY>=5

p = p.reshape(10000,)
p.shape




(10000,)

In [9]:
testX59 = testX[p,:,:,:] 
testX59.shape

(5000, 32, 32, 3)

In [10]:
testY59 = testY[p,:]
testY59.shape

(5000, 1)

In [11]:
p = trainY<5

p = p.reshape(50000,)
p.shape

(50000,)

In [12]:
trainX04 = trainX[p,:,:,:] 
trainX04.shape

(25000, 32, 32, 3)

In [13]:
trainY04 = trainY[p,:]
trainY04.shape

(25000, 1)

In [14]:
p = testY<5

p = p.reshape(10000,)
p.shape

(10000,)

In [15]:
testX04 = testX[p,:,:,:] 
testX04.shape

(5000, 32, 32, 3)

In [16]:
testY04 = testY[p,:]
testY04.shape

(5000, 1)

In [17]:
print(np.unique(trainY59))
print(np.unique(testY59))
print(np.unique(trainY04))
print(np.unique(testY04))

[5 6 7 8 9]
[5 6 7 8 9]
[0 1 2 3 4]
[0 1 2 3 4]


## **The data has been separated as per the class. 0-4 has been separated in to a separate set of training and testing sets. The values with classes 5-9 have been separated to a separate training and testing sets.**

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [18]:
train_y04 = tf.keras.utils.to_categorical(trainY04, num_classes=5)
test_y04 = tf.keras.utils.to_categorical(testY04, num_classes=5)
train_y04.shape

(25000, 5)

In [19]:

oh = OneHotEncoder()
train_y59 = oh.fit_transform(trainY59)
test_y59 = oh.transform(testY59)
train_y59.shape

(25000, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

## **Model is now being built**

In [0]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()

#normalize data
model.add(BatchNormalization(input_shape=(32,32,3,)))

#Add Conv Layer
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(BatchNormalization(input_shape=(32,32,3,)))

#Add Conv Layer
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))

#Add Max Pool layer
model.add(MaxPool2D(pool_size=(2,2),strides = 2))

#normalize data
model.add(BatchNormalization(input_shape=(32,32,3,)))

#Add Dense Layers after flattening the data
model.add(Flatten())
model.add(Dense(128, activation='relu'))




#Add Dropout
model.add(Dropout(0.25))

#Add Output Layer
model.add(Dense(5, activation='softmax'))


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 256)       295168    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 256)       1024      
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0

In [0]:
#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('first_model.h5', 
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose=1)

In [28]:
#Train the model
model.fit(trainX04,train_y04,          
          validation_data=(testX04,test_y04),
          epochs=20,
          
          callbacks = [model_checkpoint],
          batch_size=32,
          )

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
24928/25000 [============================>.] - ETA: 0s - loss: 0.1579 - accuracy: 0.9436
Epoch 00001: val_accuracy improved from -inf to 0.79780, saving model to first_model.h5
25000/25000 [==============================] - 7s 294us/sample - loss: 0.1580 - accuracy: 0.9436 - val_loss: 1.0356 - val_accuracy: 0.7978
Epoch 2/20
24832/25000 [============================>.] - ETA: 0s - loss: 0.1357 - accuracy: 0.9516
Epoch 00002: val_accuracy did not improve from 0.79780
25000/25000 [==============================] - 7s 293us/sample - loss: 0.1355 - accuracy: 0.9517 - val_loss: 1.0381 - val_accuracy: 0.7910
Epoch 3/20
24832/25000 [============================>.] - ETA: 0s - loss: 0.1341 - accuracy: 0.9525
Epoch 00003: val_accuracy did not improve from 0.79780
25000/25000 [==============================] - 7s 291us/sample - loss: 0.1345 - accuracy: 0.9523 - val_loss: 0.9531 - val_accuracy: 0.7948
Epoch 4/20
24928/25000 [============

## **The best model is saved as "first_model.h5". The accuracy of the model is 80%. This model will now be loaded and the weights of the Convulational Layers will be "Transfered" to the new model.** 

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
new_model = load_model('first_model.h5')

In [31]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 256)       295168    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 256)       1024      
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0

## **The last layer of the model loaded is a softmax layer. That has to be removed before new layers can ba added to improve vaidation accuracy.**

In [32]:
new_model.pop()
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 256)       295168    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 256)       1024      
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0

In [0]:
for layer in new_model.layers[:7]:
  layer.trainable = False

## **With the last dense layer (softmax) removed, the Convolutional layers set to non trainable, we now add new layers to the loaded model.**

In [0]:
new_model.add(Dense(128, activation='relu'))
#Add Dropout
new_model.add(Dropout(0.25))

#Add Output Layer
new_model.add(Dense(5, activation='softmax'))

In [0]:
#Specify Loass and Optimizer
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('second_model.h5', 
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose=1)

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [38]:
#Train the model
new_model.fit(trainX59,train_y59.todense(),          
          validation_data=(testX59,test_y59.todense()),
          epochs=20,
          callbacks = [model_checkpoint],
          
          batch_size=32,
          )

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
24864/25000 [============================>.] - ETA: 0s - loss: 1.6540 - accuracy: 0.5272
Epoch 00001: val_accuracy improved from -inf to 0.73520, saving model to second_model.h5
25000/25000 [==============================] - 5s 214us/sample - loss: 1.6502 - accuracy: 0.5280 - val_loss: 0.8611 - val_accuracy: 0.7352
Epoch 2/20
24896/25000 [============================>.] - ETA: 0s - loss: 0.8070 - accuracy: 0.7193
Epoch 00002: val_accuracy improved from 0.73520 to 0.80980, saving model to second_model.h5
25000/25000 [==============================] - 5s 195us/sample - loss: 0.8066 - accuracy: 0.7195 - val_loss: 0.6467 - val_accuracy: 0.8098
Epoch 3/20
24992/25000 [============================>.] - ETA: 0s - loss: 0.5775 - accuracy: 0.7915
Epoch 00003: val_accuracy improved from 0.80980 to 0.83220, saving model to second_model.h5
25000/25000 [==============================] - 5s 195us/sample - loss: 0.5775 - accuracy: 0.7915 - v

## **After transfer learning, the best model is saved in "second_model.h5". The validation accuracy of this saved model is 87.14%.**

# Text classification using TF-IDF

In [0]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn import metrics


import warnings
warnings.filterwarnings('ignore')

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
y_train = twenty_train.target

In [52]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [0]:
data_train = twenty_train.data

In [0]:
y_test = twenty_test.target

In [0]:
data_test = twenty_test.data

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
# create a document-term matrix using TF-IDF
vect = TfidfVectorizer(stop_words='english')
dtm_train = vect.fit_transform(data_train)
dtm_test = vect.transform(data_test)


In [70]:
dtm_train.shape

(2257, 35482)

In [71]:
dtm_test.shape

(1502, 35482)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [72]:
logreg = LogisticRegression(C=1e9)
logreg.fit(dtm_train, y_train)
y_pred_class = logreg.predict(dtm_test)
print("Test accuracy score", metrics.accuracy_score(y_test, y_pred_class))
y_pred_class = logreg.predict(dtm_train)
print("Train accuracy score", metrics.accuracy_score(y_train, y_pred_class))

Test accuracy score 0.9174434087882823
Train accuracy score 1.0
